In [29]:
import pandas as pd
import time
from flask import current_app
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import numpy as np
import math
import os

In [30]:
# Add working directory
os.chdir("C:/Users/Swati Arora/Data Science using Python/Movie Recommender System- Pybots/Data")

In [31]:
# Read files

In [32]:
Ratings=pd.read_csv("ratings.dat",sep='::', encoding='latin-1',names=['UserID','MovieID','Rating','Timestamp'])
Movies=pd.read_csv('movies.dat',sep='::',encoding='ANSI',names=['MovieID','Title','Genres'])

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [33]:
Movies.head()


,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [34]:
#Movies.at[988,'Genres']='None'

In [35]:
len_df= len(Movies['Genres'])

In [36]:
#Splitting the names 
df1=Movies['Genres'].str.split('|', expand=True)
df1.head()

,0,1,2,3,4,5
0,Animation,Children's,Comedy,None,None,None
1,Adventure,Children's,Fantasy,None,None,None
2,Comedy,Romance,None,None,None,None
3,Comedy,Drama,None,None,None,None
4,Comedy,None,None,None,None,None


In [37]:
df1.shape[1]

6

In [38]:
#Putting all the things in a list 
my_list = []

for j in range(0,df1.shape[1]):
    for i in range(0,len_df):
        my_list.append(str(df1[j][i]))
        

In [39]:
import nltk

In [40]:
#Generating unique list 
unique_list = np.unique(my_list)
unique_list

array(['Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
       'Mystery', 'None', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'], 
      dtype='<U11')

In [41]:
len(unique_list)

19

In [42]:
unique_list=unique_list.tolist()

In [43]:
#Remove None 
unique_list.remove('None')

In [44]:
#Create separate column 
for i in range(0,len(unique_list)):
    a = unique_list[i]
    Movies[a]=Movies.apply(lambda _: '', axis=1)

In [45]:
unique_list

['Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [46]:
df=Movies

In [47]:
df.head()

,MovieID,Title,Genres,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Animation|Children's|Comedy,,,,,,,,...,,,,,,,,,,
1,2,Jumanji (1995),Adventure|Children's|Fantasy,,,,,,,,...,,,,,,,,,,
2,3,Grumpier Old Men (1995),Comedy|Romance,,,,,,,,...,,,,,,,,,,
3,4,Waiting to Exhale (1995),Comedy|Drama,,,,,,,,...,,,,,,,,,,
4,5,Father of the Bride Part II (1995),Comedy,,,,,,,,...,,,,,,,,,,


In [48]:
#drop title and year 
df.drop(['Title'],axis=1,inplace=True)

In [49]:
df.head()


,MovieID,Genres,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Animation|Children's|Comedy,,,,,,,,,,,,,,,,,,
1,2,Adventure|Children's|Fantasy,,,,,,,,,,,,,,,,,,
2,3,Comedy|Romance,,,,,,,,,,,,,,,,,,
3,4,Comedy|Drama,,,,,,,,,,,,,,,,,,
4,5,Comedy,,,,,,,,,,,,,,,,,,


In [50]:
import re

In [51]:
# Puttonh teh dummy variable 
for i in range(2,df.shape[1]):
    for j in range(df.shape[0]):
      # if bool(re.search(df.columns[i],df.ix[j,'genres'])):  
            #df.iat[j,i]=1
        #else df.iat[j,i]=0
        a =str(df.columns[i])
        #print(i,j)
        if bool(re.search(a,df.ix[j,'Genres'])):
            df.iat[j,i]=1
        else:
            df.iat[j,i]=0
           

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  if __name__ == '__main__':


In [52]:
df.head()

,MovieID,Genres,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Animation|Children's|Comedy,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,Adventure|Children's|Fantasy,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,3,Comedy|Romance,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,4,Comedy|Drama,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Comedy,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [53]:
df.drop('Genres',axis=1,inplace=True)

In [54]:
df=df.rename(columns = {'movieId':'MovieID'})

In [55]:
#Saving the file to csv
df.to_csv('genre_matrix.csv',encoding='utf-8')

In [56]:
df.head()

,MovieID,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,4,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
